# 문장 사이의 거리 - 유사도

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
vectorizer = CountVectorizer(min_df=1)

## 훈련용 문장

In [5]:
contents = ['상처받은 아이들은 너무 일찍 커버려',
            '내가 상처받은 거 아는 사람 불편해',
            '잘 사는 사람들은 좋은 사람 되기 쉬워',
            '아무 일도 아니야 괜찮아']

In [6]:
from konlpy.tag import Twitter
t = Twitter()

/home/in/venv/nltk/lib/python3.8/site-packages/konlpy/tag/_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 형태소 분석

In [7]:
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['상처', '받은', '아이', '들', '은', '너무', '일찍', '커버', '려'],
 ['내', '가', '상처', '받은', '거', '아는', '사람', '불편해'],
 ['잘', '사는', '사람', '들', '은', '좋은', '사람', '되기', '쉬워'],
 ['아무', '일도', '아니야', '괜찮아']]

In [8]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    contents_for_vectorize.append(sentence)

contents_for_vectorize

[' 상처 받은 아이 들 은 너무 일찍 커버 려',
 ' 내 가 상처 받은 거 아는 사람 불편해',
 ' 잘 사는 사람 들 은 좋은 사람 되기 쉬워',
 ' 아무 일도 아니야 괜찮아']

## vectorize

In [9]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<4x17 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [10]:
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

In [11]:
vectorizer.get_feature_names()

/home/in/venv/nltk/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['괜찮아',
 '너무',
 '되기',
 '받은',
 '불편해',
 '사는',
 '사람',
 '상처',
 '쉬워',
 '아는',
 '아니야',
 '아무',
 '아이',
 '일도',
 '일찍',
 '좋은',
 '커버']

In [16]:
X.toarray().transpose()

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 2, 0],
       [1, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]])

## 테스트용 문장

In [17]:
new_post = ['상처받기 싫어 괜찮아']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 상처 받기 싫어 괜찮아']

In [18]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

## 유클리드 거리

In [19]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [20]:
dist = [dist_raw(each, new_post_vec) for each in X]
dist

[2.449489742783178, 2.23606797749979, 3.1622776601683795, 2.0]

In [21]:
print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('test post is --> ', new_post)
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  2.0
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is -->  아무 일도 아니야 괜찮아


## 유사도 - 벡터 사이의 거리
* 결국 중요한건 벡터로 잘 만드는 것
* & 만들어진 벡터 사이의 거리를 잘 계산하는 것

In [22]:
for i in range(0, len(contents)):
    print(X.getrow(i).toarray())

print('------------------------')
print(new_post_vec.toarray())

[[0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 1]]
[[0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0]]
[[0 0 1 0 0 1 2 0 1 0 0 0 0 0 0 1 0]]
[[1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0]]
------------------------
[[1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


# TF-IDF

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

In [26]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(4, 17)

In [27]:
X.toarray().transpose()

array([[0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.40104275, 0.6191303 , 0.        ],
       [0.34431452, 0.40104275, 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.        , 0.50867187, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.5       ],
       [0.43671931, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.39264414, 0.        ],
       [0.43671931, 0.        , 0.        , 0.        ]])

## normalize - 거리를 구하는 다른 방법

In [28]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())

    delta = v1_normalized - v2_normalized

    return sp.linalg.norm(delta.toarray())

In [29]:
dist = [dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('test post is --> ', new_post)
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  1.137054624375387
test post is -->  ['상처받기 싫어 괜찮아']
best dist post is -->  아무 일도 아니야 괜찮아


# Naver API 이용해서 문장 처리

In [30]:
import urllib.request
import json
import datetime

In [31]:
def gen_search_url(api_node, search_text, start_num, disp_num):
    base = 'https://openapi.naver.com/v1/search'
    node = '/' + api_node + '.json'
    param_query = '?query=' + urllib.parse.quote(search_text)
    param_start = '&start=' + str(start_num)
    param_disp = '&display=' + str(disp_num)

    return base + node + param_query + param_start + param_disp

In [32]:
def get_result_onpage(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)

    print("[%s] Url Request Success" % datetime.datetime.now())
    
    return json.loads(response.read().decode("utf-8"))


In [35]:
client_id = 'cBndwqBBOq6XvgRIK5yf'
client_secret = '_CAloMhNJh'

url = gen_search_url('kin', '파이썬', 10, 10)
one_result = get_result_onpage(url)
one_result

[2022-11-21 14:07:34.446281] Url Request Success


{'lastBuildDate': 'Mon, 21 Nov 2022 14:07:34 +0900',
 'total': 248908,
 'start': 10,
 'display': 10,
 'items': [{'title': 'Python <b>파이썬</b> pip 설치 오류',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=104&docId=430026279&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=10&search_sort=0&spq=0',
   'description': '<b>파이썬</b> pip 설치하려고 curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py 입력하고 다운로드까지 되는거... 알려주세요  <b>파이썬</b> 실행했을 때 &quot;Python&quot;으로만 출력되는 것은, <b>파이썬</b>이 제대로 설치가 안됐다는... '},
  {'title': '<b>파이썬</b> 적분계산',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=432715928&qb=7YyM7J207I2s&enc=utf8&section=kin.qna&rank=11&search_sort=0&spq=0',
   'description': '<b>파이썬</b>으로 적분을 계산하는 프로그램을 만들려고 하는데 input 함수를 사용해서 피적분함수를 입력받을... <b>파이썬</b>에서 적분이든 일련의 수식을 계산할 때 lamba 함수를 많이 사용합니다. fx = lambda x : x * 3 + 1 fx(5)... '},
  {'title': '비쥬얼 스튜디오 <b>파이썬</b>',
   'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=104&docId=430635427&qb=7Y

In [36]:
len(one_result)

5

In [37]:
one_result['items'][0]['description']

'<b>파이썬</b> pip 설치하려고 curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py 입력하고 다운로드까지 되는거... 알려주세요  <b>파이썬</b> 실행했을 때 &quot;Python&quot;으로만 출력되는 것은, <b>파이썬</b>이 제대로 설치가 안됐다는... '

## 태그 제거

In [38]:
def delete_tag(input_str):
    input_str = input_str.replace('<b>', '')
    input_str = input_str.replace('</b>', '')

    return input_str

In [39]:
def get_description(pages):
    contents = []
    for sentences in pages['items']:
        contents.append(delete_tag(sentences['description']))

    return contents

## 컨텐츠를 list로

In [40]:
contents = get_description(one_result)
contents

['파이썬 pip 설치하려고 curl https://bootstrap.pypa.io/get-pip.py -o get-pip.py 입력하고 다운로드까지 되는거... 알려주세요  파이썬 실행했을 때 &quot;Python&quot;으로만 출력되는 것은, 파이썬이 제대로 설치가 안됐다는... ',
 '파이썬으로 적분을 계산하는 프로그램을 만들려고 하는데 input 함수를 사용해서 피적분함수를 입력받을... 파이썬에서 적분이든 일련의 수식을 계산할 때 lamba 함수를 많이 사용합니다. fx = lambda x : x * 3 + 1 fx(5)... ',
 '파이썬하고 셀레니움 이용해서 자동으로 유튜브 영상 트는코드를 작성했는데 터미널에서 실행시키는법을... ctrl + F5 해서 실행하시면 될꺼같습니다. 터미널에서 실행한다고 하신다면, python 파이썬파일.py 로 실행하시면 됩니다. ',
 '... 사람은 파이썬을 써서 둘 중 어떤걸 써야할지, 두 프로그램의 쓰임새는 무엇인지 (예: 파이썬은 ~~개발할때... 하고 싶어서 여쭈어봅니다 ㅠㅠ vscode는 소스 코드 편집기이고 소스코드에는 파이썬 자바 등등이 되겠습니다~ ',
 '현재 코랩으로 파이썬 데이터 분석중인데 .query()만 작성하면 이런 오류가 뜨는데 문제가 뭘까요..? 제가... 오류: Python keyword not valid identifier in numexpr query (mpg는 데이터프레임 명입니다.) class는 파이썬... ',
 '현재 파이썬 기초 끝내고 진로를 프로그래밍 쪽으로 생각 중인 중3인데요, 현재... 현재 파이썬을 배우면서도 이걸 게임을 만들 때 어느 정도까지 써먹을 수... 파이썬은 언어이고 유니티는 툴?이니까 언어인 파이썬을 사용해여 유니티로... ',
 '친구가 파이썬 어쩌고저쩌고 해서 카톡으로 ip털고 비번 턴다고 협박하는데... 파이썬으로 만든 프로그램을 실행시키게 하면 링크접속으로 리다이렉트... 파이썬은 프로그램을 개발하기 위한 언어입니다. 창의력만 있다면 어

## 형태소 분석

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

t = Okt()
vectorizer = CountVectorizer(min_df = 1)

In [45]:
contents_tokens = [t.morphs(row) for row in contents]
contents_tokens

[['파이썬',
  'pip',
  '설치',
  '하려고',
  'curl',
  'https://bootstrap.pypa.io/get-pip.py',
  '-',
  'o',
  'get',
  '-',
  'pip',
  '.',
  'py',
  '입력',
  '하고',
  '다운로드',
  '까지',
  '되는거',
  '...',
  '알려주세요',
  '파이썬',
  '실행',
  '했을',
  '때',
  '&',
  'quot',
  ';',
  'Python',
  '&',
  'quot',
  ';',
  '으로만',
  '출력',
  '되는',
  '것',
  '은',
  ',',
  '파이썬',
  '이',
  '제대로',
  '설치',
  '가',
  '안',
  '됐다는',
  '...'],
 ['파이썬',
  '으로',
  '적분',
  '을',
  '계산',
  '하는',
  '프로그램',
  '을',
  '만들려고',
  '하는데',
  'input',
  '함수',
  '를',
  '사용',
  '해서',
  '피',
  '적',
  '분함수를',
  '입력',
  '받을',
  '...',
  '파이썬',
  '에서',
  '적분',
  '이든',
  '일련',
  '의',
  '수식',
  '을',
  '계산',
  '할',
  '때',
  'lamba',
  '함수',
  '를',
  '많이',
  '사용',
  '합니다',
  '.',
  'fx',
  '=',
  'lambda',
  'x',
  ':',
  'x',
  '*',
  '3',
  '+',
  '1',
  'fx',
  '(',
  '5',
  ')...'],
 ['파이썬',
  '하고',
  '셀레니움',
  '이용',
  '해서',
  '자동',
  '으로',
  '유튜브',
  '영상',
  '트는',
  '코드',
  '를',
  '작성',
  '했는데',
  '터미널',
  '에서',
  '실행',
  '시키는',
  '법',
  '을',
 

In [47]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word

    contents_for_vectorize.append(sentence)

contents_for_vectorize

[' 파이썬 pip 설치 하려고 curl https://bootstrap.pypa.io/get-pip.py - o get - pip . py 입력 하고 다운로드 까지 되는거 ... 알려주세요 파이썬 실행 했을 때 & quot ; Python & quot ; 으로만 출력 되는 것 은 , 파이썬 이 제대로 설치 가 안 됐다는 ...',
 ' 파이썬 으로 적분 을 계산 하는 프로그램 을 만들려고 하는데 input 함수 를 사용 해서 피 적 분함수를 입력 받을 ... 파이썬 에서 적분 이든 일련 의 수식 을 계산 할 때 lamba 함수 를 많이 사용 합니다 . fx = lambda x : x * 3 + 1 fx ( 5 )...',
 ' 파이썬 하고 셀레니움 이용 해서 자동 으로 유튜브 영상 트는 코드 를 작성 했는데 터미널 에서 실행 시키는 법 을 ... ctrl + F 5 해서 실행 하시면 될꺼 같습니다 . 터미널 에서 실행 한다고 하신다면 , python 파이썬 파일 . py 로 실행 하시면 됩니다 .',
 ' ... 사람 은 파이썬 을 써서 둘 중 어떤 걸 써야 할지 , 두 프로그램 의 쓰임 새는 무엇 인지 ( 예 : 파이썬 은 ~~ 개발 할 때 ... 하고 싶어서 여쭈어 봅니다 ㅠㅠ vscode 는 소스 코드 편집 기이 고 소스코드 에는 파이썬 자바 등등 이 되겠습니다 ~',
 ' 현재 코랩 으로 파이썬 데이터 분석 중 인데 . query () 만 작성 하면 이런 오류 가 뜨는데 문제 가 뭘 까요 ..? 제 가 ... 오류 : Python keyword not valid identifier in numexpr query ( mpg 는 데이터 프레임 명 입니다 .) class 는 파이썬 ...',
 ' 현재 파이썬 기초 끝내고 진로 를 프로그래밍 쪽 으로 생각 중인 중 3 인데요 , 현재 ... 현재 파이썬 을 배우면서도 이 걸 게임 을 만들 때 어느 정도 까지 써 먹을 수 ... 파이썬 은 언어 이고 유니티 는 툴 ? 이니까 언어 인 파이썬 을 사용 해여 유니

In [48]:
X = vectorizer.fit_transform(contents_for_vectorize)
X

<10x205 sparse matrix of type '<class 'numpy.int64'>'
	with 254 stored elements in Compressed Sparse Row format>

In [49]:
num_samples, num_features = X.shape
num_samples, num_features

(10, 205)

In [50]:
vectorizer.get_feature_names()

/home/in/venv/nltk/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['10',
 '30',
 'append',
 'bootstrap',
 'break',
 'class',
 'ctrl',
 'curl',
 'for',
 'for문',
 'from',
 'fx',
 'get',
 'https',
 'identifier',
 'if',
 'import',
 'in',
 'input',
 'io',
 'ip',
 'keyword',
 'lamba',
 'lambda',
 'len',
 'move',
 'mpg',
 'not',
 'num',
 'numexpr',
 'ocr',
 'pip',
 'py',
 'pypa',
 'python',
 'query',
 'quot',
 'randint',
 'random',
 'true',
 'valid',
 'vscode',
 'while',
 'while문',
 'ㅜㅜ',
 'ㅠㅠ',
 '감사합니다',
 '같습니다',
 '개발',
 '게임',
 '경험',
 '계산',
 '고수',
 '과제',
 '그대로',
 '기이',
 '기초',
 '까요',
 '까지',
 '까지는',
 '끝내고',
 '내공',
 '내용',
 '넣기',
 '다운로드',
 '데이터',
 '도와주세요',
 '도움',
 '됐다는',
 '되겠습니다',
 '되는',
 '되는거',
 '될꺼',
 '됩니다',
 '두자',
 '드릴게요',
 '들께',
 '등등',
 '뜨는데',
 '리다이렉트',
 '리수',
 '링크',
 '만든',
 '만들',
 '만들려고',
 '많이',
 '먹을',
 '무엇',
 '문제',
 '바랍니다',
 '받을',
 '방식',
 '배우면서도',
 '봅니다',
 '분석',
 '분함수를',
 '비번',
 '사람',
 '사용',
 '새는',
 '생각',
 '설치',
 '셀레니움',
 '소스',
 '소스코드',
 '손글씨',
 '수식',
 '시키게',
 '시키는',
 '실행',
 '싶어서',
 '써서',
 '써야',
 '쓰임',
 '아니라',
 '아래',
 '아시',
 '안녕하세요',
 '알려주세요',
 '어느',
 '어

In [52]:
new_post = ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word

    new_post_for_vectorize.append(sentence)

new_post_for_vectorize

[' 파이썬 을 배우는데 좋은 방법 이 어떤 것 인지 추천 해주세요']

In [53]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]])

## 유클리드 거리

In [55]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

In [56]:
dist = [dist_raw(each, new_post_vec) for each in X]
dist

[7.14142842854285,
 6.4031242374328485,
 7.483314773547883,
 5.291502622129181,
 6.082762530298219,
 6.855654600401044,
 6.557438524302,
 5.830951894845301,
 5.744562646538029,
 7.0]

In [57]:
print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('test post is --> ', new_post)
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  5.291502622129181
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  ... 사람은 파이썬을 써서 둘 중 어떤걸 써야할지, 두 프로그램의 쓰임새는 무엇인지 (예: 파이썬은 ~~개발할때... 하고 싶어서 여쭈어봅니다 ㅠㅠ vscode는 소스 코드 편집기이고 소스코드에는 파이썬 자바 등등이 되겠습니다~ 


## normalize로 거리 구하기

In [58]:
dist = [dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('test post is --> ', new_post)
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  1.011978224593527
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  ... 사람은 파이썬을 써서 둘 중 어떤걸 써야할지, 두 프로그램의 쓰임새는 무엇인지 (예: 파이썬은 ~~개발할때... 하고 싶어서 여쭈어봅니다 ㅠㅠ vscode는 소스 코드 편집기이고 소스코드에는 파이썬 자바 등등이 되겠습니다~ 


## TF-IDF

In [59]:
def tfidf(t, d, D):
    tf = float(d.count(t)) / sum(d.count(w) for w in set(d))
    idf = sp.log(float(len(D)) / len([doc for doc in D if t in doc]))
    return tf * idf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=1, decode_error='ignore')

In [60]:
X = vectorizer.fit_transform(contents_for_vectorize)
num_samples, num_features = X.shape
num_samples, num_features

(10, 205)

In [62]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)
new_post_vec.toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0]])

In [63]:
dist = [dist_norm(each, new_post_vec) for each in X]

print('Best post is ', dist.index(min(dist)), ', dist = ', min(dist))
print('test post is --> ', new_post)
print('best dist post is --> ', contents[dist.index(min(dist))])

Best post is  3 , dist =  1.011978224593527
test post is -->  ['파이썬을 배우는데 좋은 방법이 어떤 것인지 추천해주세요']
best dist post is -->  ... 사람은 파이썬을 써서 둘 중 어떤걸 써야할지, 두 프로그램의 쓰임새는 무엇인지 (예: 파이썬은 ~~개발할때... 하고 싶어서 여쭈어봅니다 ㅠㅠ vscode는 소스 코드 편집기이고 소스코드에는 파이썬 자바 등등이 되겠습니다~ 
